# Preprocessing the data

### Loading required packages

In [4]:
import pandas as pd
import numpy as np
import boto3
from datetime import datetime

### Loading and combining data
#### Meteo data

In [ ]:
# loading meteo data
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
years = ['2022']
base_url_meteo = 'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_{}{}.csv'

dfs = []

for year in years:
    for quarter in quarters:
        url = base_url_meteo.format(year, quarter)
        df = pd.read_csv(url)
        dfs.append(df)

# Concatenate all the dataframes into a single dataframe
meteo_combined_df = pd.concat(dfs, ignore_index=True)
meteo_combined_df.head()

del dfs # deleting the individual meteo datasets


In [ ]:
# Drop the columns we won't use
columns_to_keep = ['LC_RAININ', 'LC_DAILYRAIN', 'LC_WINDDIR', 'LC_WINDSPEED', 'LC_TEMP_QCL3', 'Month', 'Day', 'Hour']  #there's less columns we keep than drop
columns_to_drop = set(meteo_combined_df.columns) - set(columns_to_keep)
meteo_combined_df.drop(columns=columns_to_drop, inplace=True)
meteo_combined_df.head()

In [ ]:
# check for percentage of missing values in each column
print(meteo_combined_df.isnull().sum() / len(meteo_combined_df))

In [ ]:
# Forward fill missing values in the original DataFrame
meteo_combined_df.ffill(inplace=True)

# check whether there are missing values left
print(meteo_combined_df.isnull().sum())

In [ ]:
# calculate summary statistics for daily rain sum
summary_stats = meteo_combined_df['LC_DAILYRAIN'].describe()
print(summary_stats)

In [ ]:
# Count the number of non-zero values in the 'LC_DAILYRAIN' column
nonzero_count = np.count_nonzero(meteo_combined_df['LC_DAILYRAIN'])

# Display the fraction of non-zero values
print("Fraction of non-zero values:", nonzero_count/len(meteo_combined_df))

In [ ]:
# Create dataframe per hour

# Specify the aggregation function for each column
  # for LC_DAILYRAIN we take the last value because it's cumulative, for other columns the mean
aggregations = {
    'LC_DAILYRAIN': 'mean',  # Select the last value for 'LC_DAILYRAIN' ###TAKE MEAN FOR NOW TO MAKE THE GRAPHS LOOK OK
    'LC_RAININ': 'mean',  
    'LC_WINDDIR': 'mean',
    'LC_WINDDIR': 'mean', 
    'LC_WINDSPEED': 'mean', 
    'LC_TEMP_QCL3': 'mean'
}

# Perform the groupby aggregation
meteo_per_hour = meteo_combined_df.groupby(['Month', 'Day', 'Hour']).mean()
meteo_per_hour = meteo_per_hour.reset_index()
meteo_per_hour.head()

In [ ]:
# Create dataframe per day

# still the same "aggregations" as before
meteo_per_day = meteo_combined_df.groupby(['Month', 'Day']).mean()
meteo_per_day = meteo_per_day.reset_index()
meteo_per_day.head()

In [ ]:
# Create dataframe per month

# still the same "aggregations" as before
meteo_per_month = meteo_combined_df.groupby(['Month']).mean()
meteo_per_month = meteo_per_month.reset_index()
meteo_per_month.head()

In [ ]:
# delete combined dataframe
del meteo_combined_df

In [ ]:
# export dataframes (only needs to be ran once so comment it out)
# meteo_per_hour.to_csv('hourly_weatherdata_2022.csv', index=False)
# meteo_per_day.to_csv('daily_weatherdata_2022.csv', index=False)
# meteo_per_month.to_csv('monthly_weatherdata_2022.csv', index=False)

#### Noise data

- January 

In [5]:
# Define a list of URLs
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

In [6]:
# Combining the datasets for January
combined_jan = pd.concat(dfs, ignore_index=True)
print(combined_jan.head())

del dfs # deleting the separate dataframes to minimize memory usage

  #object_id                   description         result_timestamp  lamax  \
0     255441  MP 03: Naamsestraat 62 Taste  01/01/2022 00:00:00.349   87.6   
1     255441  MP 03: Naamsestraat 62 Taste  01/01/2022 00:00:01.349   84.5   
2     255441  MP 03: Naamsestraat 62 Taste  01/01/2022 00:00:02.349   84.8   
3     255441  MP 03: Naamsestraat 62 Taste  01/01/2022 00:00:03.349   81.9   
4     255441  MP 03: Naamsestraat 62 Taste  01/01/2022 00:00:04.349   78.3   

  lamax_unit  laeq laeq_unit   lceq lceq_unit  lcpeak lcpeak_unit  
0      dB(A)  82.7     dB(A)  83.61     dB(C)   97.17       dB(C)  
1      dB(A)  83.1     dB(A)  84.42     dB(C)   96.41       dB(C)  
2      dB(A)  82.7     dB(A)  84.19     dB(C)   96.24       dB(C)  
3      dB(A)  79.3     dB(A)  81.08     dB(C)   94.03       dB(C)  
4      dB(A)  76.0     dB(A)  77.12     dB(C)   89.81       dB(C)  


In [7]:
# extract the month, day, hour, minute of "result_timestamp"
combined_jan['result_timestamp'] = pd.to_datetime(combined_jan['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
combined_jan['month'] = combined_jan['result_timestamp'].dt.month
combined_jan['day'] = combined_jan['result_timestamp'].dt.day
combined_jan['hour'] = combined_jan['result_timestamp'].dt.hour
#combined_jan['minute'] = combined_jan['result_timestamp'].dt.minute

combined_jan.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,month,day,hour
0,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,dB(C),1,1,0
1,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,dB(C),1,1,0
2,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,dB(C),1,1,0
3,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,dB(C),1,1,0
4,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,dB(C),1,1,0


In [8]:
# Drop the columns we won't use
columns_to_keep = ['description', 'lamax', 'laeq', 'month', 'day', 'hour'] #also minute if we calculate it
columns_to_drop = set(combined_jan.columns) - set(columns_to_keep)
combined_jan.drop(columns=columns_to_drop, inplace=True)
combined_jan.head()

,description,lamax,laeq,month,day,hour
0,MP 03: Naamsestraat 62 Taste,87.6,82.7,1,1,0
1,MP 03: Naamsestraat 62 Taste,84.5,83.1,1,1,0
2,MP 03: Naamsestraat 62 Taste,84.8,82.7,1,1,0
3,MP 03: Naamsestraat 62 Taste,81.9,79.3,1,1,0
4,MP 03: Naamsestraat 62 Taste,78.3,76.0,1,1,0


In [9]:
# check for missing values in each column
print(combined_jan.isnull().sum())

description    0
lamax          3
laeq           3
month          0
day            0
hour           0
dtype: int64


In [10]:
# forward fill missing values 
combined_jan.ffill(inplace=True)

# check whether there are missing values left
print(combined_jan.isnull().sum())

description    0
lamax          0
laeq           0
month          0
day            0
hour           0
dtype: int64


In [11]:
# Create dataframe per hour
jan_per_hour = combined_jan.groupby(['month', 'day', 'hour', 'description']).mean()
jan_per_hour = jan_per_hour.reset_index()
jan_per_hour.head()

,month,day,hour,description,lamax,laeq
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806


In [12]:
# Create dataframe per day
combined_jan.drop('hour', axis=1, inplace=True)
jan_per_day= combined_jan.groupby(['month', 'day', 'description']).mean()
jan_per_day = jan_per_day.reset_index()
jan_per_day.head()

,month,day,description,lamax,laeq
0,1,1,MP 03: Naamsestraat 62 Taste,51.665242,49.992637
1,1,1,MP 05: Calvariekapel KU Leuven,48.747476,46.504067
2,1,1,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220
3,1,1,MP 07: Naamsestraat 81,45.908501,44.373056
4,1,2,MP 03: Naamsestraat 62 Taste,51.407297,50.094018


In [13]:
# Create dataframe per month
combined_jan.drop('day', axis=1, inplace=True)
jan_per_month = combined_jan.groupby(['month', 'description']).mean()
jan_per_month = jan_per_month.reset_index()
jan_per_month.head()

,month,description,lamax,laeq
0,1,MP 03: Naamsestraat 62 Taste,53.239009,51.727544
1,1,MP 05: Calvariekapel KU Leuven,50.374792,48.560197
2,1,MP 06: Parkstraat 2 La Filosovia,50.086348,48.274795
3,1,MP 07: Naamsestraat 81,48.800443,47.515371


In [14]:
del combined_jan

- February

In [15]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

In [16]:
# Combining the datasets for February
combined_feb = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the month, day, hour, minute of "result_timestamp"
combined_feb['result_timestamp'] = pd.to_datetime(combined_feb['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
combined_feb['month'] = combined_feb['result_timestamp'].dt.month
combined_feb['day'] = combined_feb['result_timestamp'].dt.day
combined_feb['hour'] = combined_feb['result_timestamp'].dt.hour
#combined_feb['minute'] = combined_feb['result_timestamp'].dt.minute

combined_feb.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,month,day,hour
0,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:26.734,72.5,dB(A),71.8,dB(A),80.80,dB(C),92.17,dB(C),2,28,8
1,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:27.734,71.1,dB(A),68.8,dB(A),78.96,dB(C),90.57,dB(C),2,28,8
2,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:28.734,72.0,dB(A),70.3,dB(A),78.64,dB(C),91.05,dB(C),2,28,8
3,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:29.734,69.2,dB(A),68.5,dB(A),76.94,dB(C),88.43,dB(C),2,28,8
4,255439,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:23:30.734,70.6,dB(A),69.7,dB(A),76.75,dB(C),90.19,dB(C),2,28,8


In [17]:
# Drop the columns we won't use
columns_to_keep = ['description', 'lamax', 'laeq', 'month', 'day', 'hour'] #also minute if we calculate it
columns_to_drop = set(combined_feb.columns) - set(columns_to_keep)
combined_feb.drop(columns=columns_to_drop, inplace=True)
combined_feb.head()

,description,lamax,laeq,month,day,hour
0,MP 01: Naamsestraat 35 Maxim,72.5,71.8,2,28,8
1,MP 01: Naamsestraat 35 Maxim,71.1,68.8,2,28,8
2,MP 01: Naamsestraat 35 Maxim,72.0,70.3,2,28,8
3,MP 01: Naamsestraat 35 Maxim,69.2,68.5,2,28,8
4,MP 01: Naamsestraat 35 Maxim,70.6,69.7,2,28,8


In [18]:
# check for missing values in each column
print(combined_feb.isnull().sum())

description    0
lamax          0
laeq           0
month          0
day            0
hour           0
dtype: int64


In [19]:
# Create dataframe per hour
feb_per_hour = combined_feb.groupby(['month', 'day', 'hour', 'description']).mean()
feb_per_hour = feb_per_hour.reset_index()
print(feb_per_hour.head())

# Create dataframe per day
combined_feb.drop('hour', axis=1, inplace=True)
feb_per_day = combined_feb.groupby(['month', 'day', 'description']).mean()
feb_per_day = feb_per_day.reset_index()
print(feb_per_day.head())

# Create dataframe per month
combined_feb.drop('day', axis=1, inplace=True)
feb_per_month = combined_feb.groupby(['month', 'description']).mean()
feb_per_month = feb_per_month.reset_index()
print(feb_per_month.head())

del combined_feb

   month  day  hour                       description      lamax       laeq
0      2    1     0      MP 03: Naamsestraat 62 Taste  51.096750  48.781028
1      2    1     0    MP 05: Calvariekapel KU Leuven  48.761611  45.566611
2      2    1     0  MP 06: Parkstraat 2 La Filosovia  49.019056  45.792000
3      2    1     0            MP 07: Naamsestraat 81  43.687944  41.601778
4      2    1     1      MP 03: Naamsestraat 62 Taste  48.237583  46.278333
   month  day                       description      lamax       laeq
0      2    1      MP 03: Naamsestraat 62 Taste  54.948997  53.332087
1      2    1    MP 05: Calvariekapel KU Leuven  53.203722  51.239140
2      2    1  MP 06: Parkstraat 2 La Filosovia  53.183853  51.231713
3      2    1            MP 07: Naamsestraat 81  51.413080  50.029712
4      2    2      MP 03: Naamsestraat 62 Taste  54.271621  52.578714
   month                       description      lamax       laeq
0      2     MP 01: Naamsestraat 35  Maxim  57.287668  55.2

- March

In [20]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for March
combined_mar = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the month, day, hour, minute of "result_timestamp"
combined_mar['result_timestamp'] = pd.to_datetime(combined_mar['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
combined_mar['month'] = combined_mar['result_timestamp'].dt.month
combined_mar['day'] = combined_mar['result_timestamp'].dt.day
combined_mar['hour'] = combined_mar['result_timestamp'].dt.hour
#combined_mar['minute'] = combined_mar['result_timestamp'].dt.minute

combined_mar.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,month,day,hour
0,255439,MP 01: Naamsestraat 35 Maxim,2022-03-01 00:00:00.462,60.5,dB(A),57.9,dB(A),63.36,dB(C),76.57,dB(C),3,1,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-03-01 00:00:01.462,54.1,dB(A),53.2,dB(A),61.86,dB(C),74.52,dB(C),3,1,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-03-01 00:00:02.462,61.4,dB(A),57.5,dB(A),64.12,dB(C),76.46,dB(C),3,1,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-03-01 00:00:03.462,61.3,dB(A),59.1,dB(A),64.68,dB(C),76.67,dB(C),3,1,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-03-01 00:00:04.462,61.1,dB(A),58.4,dB(A),64.53,dB(C),77.07,dB(C),3,1,0


In [21]:
# Drop the columns we won't use
columns_to_keep = ['description', 'lamax', 'laeq', 'month', 'day', 'hour'] #also minute if we calculate it
columns_to_drop = set(combined_mar.columns) - set(columns_to_keep)
combined_mar.drop(columns=columns_to_drop, inplace=True)
combined_mar.head()

,description,lamax,laeq,month,day,hour
0,MP 01: Naamsestraat 35 Maxim,60.5,57.9,3,1,0
1,MP 01: Naamsestraat 35 Maxim,54.1,53.2,3,1,0
2,MP 01: Naamsestraat 35 Maxim,61.4,57.5,3,1,0
3,MP 01: Naamsestraat 35 Maxim,61.3,59.1,3,1,0
4,MP 01: Naamsestraat 35 Maxim,61.1,58.4,3,1,0


In [22]:
# check for missing values in each column
print(combined_mar.isnull().sum())

description     0
lamax          13
laeq           13
month           0
day             0
hour            0
dtype: int64


In [24]:
# forward fill missing values 
combined_mar.ffill(inplace=True)

# check whether there are missing values left
print(combined_mar.isnull().sum())

description    0
lamax          0
laeq           0
month          0
day            0
hour           0
dtype: int64


In [25]:
# Create dataframe per hour
mar_per_hour = combined_mar.groupby(['month', 'day', 'hour', 'description']).mean()
mar_per_hour = mar_per_hour.reset_index()
print(mar_per_hour.head())

# Create dataframe per day
combined_mar.drop('hour', axis=1, inplace=True)
mar_per_day = combined_mar.groupby(['month', 'day', 'description']).mean()
mar_per_day = mar_per_day.reset_index()
print(mar_per_day.head())

# Create dataframe per month
combined_mar.drop('day', axis=1, inplace=True)
mar_per_month = combined_mar.groupby(['month', 'description']).mean()
mar_per_month = mar_per_month.reset_index()
print(mar_per_month.head())

del combined_mar

   month  day  hour                       description      lamax       laeq
0      3    1     0     MP 01: Naamsestraat 35  Maxim  63.333278  60.815722
1      3    1     0       MP 02: Naamsestraat 57 Xior  54.192944  51.577083
2      3    1     0      MP 03: Naamsestraat 62 Taste  53.878000  51.471417
3      3    1     0    MP 05: Calvariekapel KU Leuven  53.308556  50.065111
4      3    1     0  MP 06: Parkstraat 2 La Filosovia  52.805778  49.801306
   month  day                       description      lamax       laeq
0      3    1     MP 01: Naamsestraat 35  Maxim  58.176227  55.849949
1      3    1       MP 02: Naamsestraat 57 Xior  54.176705  52.183638
2      3    1      MP 03: Naamsestraat 62 Taste  55.187625  53.303638
3      3    1    MP 05: Calvariekapel KU Leuven  54.506068  52.138010
4      3    1  MP 06: Parkstraat 2 La Filosovia  53.991586  51.884388
   month                       description      lamax       laeq
0      3     MP 01: Naamsestraat 35  Maxim  57.351576  55.0

- April

In [26]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for March
combined_apr = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the month, day, hour, minute of "result_timestamp"
combined_apr['result_timestamp'] = pd.to_datetime(combined_apr['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
combined_apr['month'] = combined_apr['result_timestamp'].dt.month
combined_apr['day'] = combined_apr['result_timestamp'].dt.day
combined_apr['hour'] = combined_apr['result_timestamp'].dt.hour
#combined_apr['minute'] = combined_apr['result_timestamp'].dt.minute

combined_apr.head()

,#object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,month,day,hour
0,255439,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:00:00.520,62.5,dB(A),59.3,dB(A),64.56,dB(C),78.13,dB(C),4,1,0
1,255439,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:00:01.520,63.3,dB(A),61.3,dB(A),65.35,dB(C),78.09,dB(C),4,1,0
2,255439,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:00:02.520,61.4,dB(A),59.1,dB(A),64.18,dB(C),76.63,dB(C),4,1,0
3,255439,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:00:03.510,58.9,dB(A),56.6,dB(A),63.58,dB(C),75.74,dB(C),4,1,0
4,255439,MP 01: Naamsestraat 35 Maxim,2022-04-01 00:00:04.510,59.6,dB(A),57.2,dB(A),63.32,dB(C),77.86,dB(C),4,1,0


In [27]:
# Drop the columns we won't use
columns_to_keep = ['description', 'lamax', 'laeq', 'month', 'day', 'hour'] #also minute if we calculate it
columns_to_drop = set(combined_apr.columns) - set(columns_to_keep)
combined_apr.drop(columns=columns_to_drop, inplace=True)
combined_apr.head()

,description,lamax,laeq,month,day,hour
0,MP 01: Naamsestraat 35 Maxim,62.5,59.3,4,1,0
1,MP 01: Naamsestraat 35 Maxim,63.3,61.3,4,1,0
2,MP 01: Naamsestraat 35 Maxim,61.4,59.1,4,1,0
3,MP 01: Naamsestraat 35 Maxim,58.9,56.6,4,1,0
4,MP 01: Naamsestraat 35 Maxim,59.6,57.2,4,1,0


In [29]:
# check for missing values in each column
print(combined_apr.isnull().sum() / len(combined_apr))

description    0.000000
lamax          0.000078
laeq           0.000002
month          0.000000
day            0.000000
hour           0.000000
dtype: float64


In [30]:
# forward fill missing values 
combined_apr.ffill(inplace=True)

# check whether there are missing values left
print(combined_apr.isnull().sum())

description    0
lamax          0
laeq           0
month          0
day            0
hour           0
dtype: int64


In [31]:
# Create dataframe per hour
apr_per_hour = combined_apr.groupby(['month', 'day', 'hour', 'description']).mean()
apr_per_hour = apr_per_hour.reset_index()
print(apr_per_hour.head())

# Create dataframe per day
combined_apr.drop('hour', axis=1, inplace=True)
apr_per_day = combined_apr.groupby(['month', 'day', 'description']).mean()
apr_per_day = apr_per_day.reset_index()
print(apr_per_day.head())

# Create dataframe per month
combined_apr.drop('day', axis=1, inplace=True)
apr_per_month = combined_apr.groupby(['month', 'description']).mean()
apr_per_month = apr_per_month.reset_index()
print(apr_per_month.head())

del combined_apr

   month  day  hour                       description      lamax       laeq
0      4    1     0     MP 01: Naamsestraat 35  Maxim  66.120833  63.600472
1      4    1     0       MP 02: Naamsestraat 57 Xior  59.498917  56.891000
2      4    1     0      MP 03: Naamsestraat 62 Taste  59.414194  56.662389
3      4    1     0    MP 05: Calvariekapel KU Leuven  58.328889  55.176444
4      4    1     0  MP 06: Parkstraat 2 La Filosovia  58.421944  55.435500
   month  day                       description      lamax       laeq
0      4    1     MP 01: Naamsestraat 35  Maxim  59.920866  57.731795
1      4    1       MP 02: Naamsestraat 57 Xior  57.569984  55.615654
2      4    1      MP 03: Naamsestraat 62 Taste  57.684719  55.881178
3      4    1    MP 05: Calvariekapel KU Leuven  57.161305  54.901119
4      4    1  MP 06: Parkstraat 2 La Filosovia  56.794676  54.744187
   month                       description      lamax       laeq
0      4     MP 01: Naamsestraat 35  Maxim  55.052381  52.6

- May

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for March
combined_may = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the month, day, hour, minute of "result_timestamp"
combined_may['result_timestamp'] = pd.to_datetime(combined_may['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
combined_may['month'] = combined_may['result_timestamp'].dt.month
combined_may['day'] = combined_may['result_timestamp'].dt.day
combined_may['hour'] = combined_may['result_timestamp'].dt.hour
#combined_may['minute'] = combined_may['result_timestamp'].dt.minute

# Drop the columns we won't use
columns_to_keep = ['description', 'lamax', 'laeq', 'month', 'day', 'hour'] #also minute if we calculate it
columns_to_drop = set(combined_may.columns) - set(columns_to_keep)
combined_may.drop(columns=columns_to_drop, inplace=True)

# check for missing values in each column
print(combined_may.isnull().sum())

In [ ]:
# Create dataframe per hour
may_per_hour = combined_may.groupby(['month', 'day', 'hour', 'description']).mean()
may_per_hour = may_per_hour.reset_index()
print(may_per_hour.head())

# Create dataframe per day
combined_may.drop('hour', axis=1, inplace=True)
may_per_day = combined_may.groupby(['month', 'day', 'description']).mean()
may_per_day = may_per_day.reset_index()
print(may_per_day.head())

# Create dataframe per month
combined_may.drop('day', axis=1, inplace=True)
may_per_month = combined_may.groupby(['month', 'description']).mean()
may_per_month = may_per_month.reset_index()
print(may_per_month.head())

del combined_may

- June

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for June
combined_jun = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_jun['result_timestamp'] = pd.to_datetime(combined_jun['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_jun['result_date'] = combined_jun['result_timestamp'].dt.date
combined_jun['result_month'] = combined_jun['result_timestamp'].dt.month
combined_jun['result_day'] = combined_jun['result_timestamp'].dt.day
combined_jun['result_hour'] = combined_jun['result_timestamp'].dt.hour
combined_jun['result_minute'] = combined_jun['result_timestamp'].dt.minute

combined_jun.head()

# aggregate the data by day
combined_jun = combined_jun.groupby(['result_date','description']).mean()
combined_jun = combined_jun.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_jun.drop(columns_to_drop, axis=1, inplace=True)
combined_jun.head(150)

# check for missing values in each column
print(combined_jun.isnull().sum())

- July

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for July
combined_jul = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_jul['result_timestamp'] = pd.to_datetime(combined_jul['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_jul['result_date'] = combined_jul['result_timestamp'].dt.date
combined_jul['result_month'] = combined_jul['result_timestamp'].dt.month
combined_jul['result_day'] = combined_jul['result_timestamp'].dt.day
combined_jul['result_hour'] = combined_jul['result_timestamp'].dt.hour
combined_jul['result_minute'] = combined_jul['result_timestamp'].dt.minute

combined_jul.head()

# aggregate the data by day
combined_jul = combined_jul.groupby(['result_date','description']).mean()
combined_jul = combined_jul.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_jul.drop(columns_to_drop, axis=1, inplace=True)
combined_jul.head(150)

# check for missing values in each column
print(combined_jul.isnull().sum())

- August

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for August
combined_aug = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_aug['result_timestamp'] = pd.to_datetime(combined_aug['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_aug['result_date'] = combined_aug['result_timestamp'].dt.date
combined_aug['result_month'] = combined_aug['result_timestamp'].dt.month
combined_aug['result_day'] = combined_aug['result_timestamp'].dt.day
combined_aug['result_hour'] = combined_aug['result_timestamp'].dt.hour
combined_aug['result_minute'] = combined_aug['result_timestamp'].dt.minute

combined_aug.head()

# aggregate the data by day
combined_aug = combined_aug.groupby(['result_date','description']).mean()
combined_aug = combined_aug.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_aug.drop(columns_to_drop, axis=1, inplace=True)
combined_aug.head(150)

# check for missing values in each column
print(combined_aug.isnull().sum())

- September

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for September
combined_sep = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_sep['result_timestamp'] = pd.to_datetime(combined_sep['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_sep['result_date'] = combined_sep['result_timestamp'].dt.date
combined_sep['result_month'] = combined_sep['result_timestamp'].dt.month
combined_sep['result_day'] = combined_sep['result_timestamp'].dt.day
combined_sep['result_hour'] = combined_sep['result_timestamp'].dt.hour
combined_sep['result_minute'] = combined_sep['result_timestamp'].dt.minute

combined_sep.head()

# aggregate the data by day
combined_sep = combined_sep.groupby(['result_date','description']).mean()
combined_sep = combined_sep.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_sep.drop(columns_to_drop, axis=1, inplace=True)
combined_sep.head(150)

# check for missing values in each column
print(combined_sep.isnull().sum())

- October

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for October
combined_oct = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_oct['result_timestamp'] = pd.to_datetime(combined_oct['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_oct['result_date'] = combined_oct['result_timestamp'].dt.date
combined_oct['result_month'] = combined_oct['result_timestamp'].dt.month
combined_oct['result_day'] = combined_oct['result_timestamp'].dt.day
combined_oct['result_hour'] = combined_oct['result_timestamp'].dt.hour
combined_oct['result_minute'] = combined_oct['result_timestamp'].dt.minute

combined_oct.head()

# aggregate the data by day
combined_oct = combined_oct.groupby(['result_date','description']).mean()
combined_oct = combined_oct.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_oct.drop(columns_to_drop, axis=1, inplace=True)
combined_oct.head(150)

# check for missing values in each column
print(combined_oct.isnull().sum())

- November

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for November
combined_nov = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_nov['result_timestamp'] = pd.to_datetime(combined_nov['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_nov['result_date'] = combined_nov['result_timestamp'].dt.date
combined_nov['result_month'] = combined_nov['result_timestamp'].dt.month
combined_nov['result_day'] = combined_nov['result_timestamp'].dt.day
combined_nov['result_hour'] = combined_nov['result_timestamp'].dt.hour
combined_nov['result_minute'] = combined_nov['result_timestamp'].dt.minute

combined_nov.head()

# aggregate the data by day
combined_nov = combined_nov.groupby(['result_date','description']).mean()
combined_nov = combined_nov.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_nov.drop(columns_to_drop, axis=1, inplace=True)
combined_nov.head(150)

# check for missing values in each column
print(combined_nov.isnull().sum())

- December

In [ ]:
# Define a list of URLs 
urls = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL and read the CSV into a DataFrame
for url in urls:
    df = pd.read_csv(url, header=0, sep=';')
    dfs.append(df)

# Now we have a list of DataFrames for each URL

# Combining the datasets for December
combined_dec = pd.concat(dfs, ignore_index=True)
del dfs # deleting the separate dataframes to minimize memory usage

# extract the date, month, hour, minute of "result_timestamp"
combined_dec['result_timestamp'] = pd.to_datetime(combined_dec['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')

combined_dec['result_date'] = combined_dec['result_timestamp'].dt.date
combined_dec['result_month'] = combined_dec['result_timestamp'].dt.month
combined_dec['result_day'] = combined_dec['result_timestamp'].dt.day
combined_dec['result_hour'] = combined_dec['result_timestamp'].dt.hour
combined_dec['result_minute'] = combined_dec['result_timestamp'].dt.minute

combined_dec.head()

# aggregate the data by day
combined_dec = combined_dec.groupby(['result_date','description']).mean()
combined_dec = combined_dec.reset_index()
columns_to_drop = ['result_hour', 'result_minute']
combined_dec.drop(columns_to_drop, axis=1, inplace=True)
combined_dec.head(150)

# check for missing values in each column
print(combined_dec.isnull().sum())

#### Yearly noise data

In [ ]:
# List of the monthly datasets
datasets = [combined_jan, combined_feb, combined_mar, combined_apr, combined_may, combined_jun, combined_jul, combined_aug, combined_sep, combined_oct, combined_nov, combined_dec]

# Concatenate the datasets vertically
combined_year = pd.concat(datasets, ignore_index=True)
del datasets

# Reset the index of the combined dataset
combined_year = combined_year.reset_index()

# Display the combined and sorted yearly dataset

combined_year.head(2000)


In [ ]:
# exporting file (only needs to be ran one time so comment it out)
# combined_year.to_csv('combined_noisedata_2022.csv', index=False)  


Now that we have exported the preprocessed dataframes for the noise and weather data of 2022, we can just use these files instead of loading all 112 files from the S3 bucket each time, as this takes a lot of time.

In [ ]:
data_noise = pd.read_csv('combined_noisedata_2022.csv', header=0, sep=',')
data_noise.head()

In [ ]:
data_weather = pd.read_csv('combined_weatherdata_2022.csv', header=0, sep=',')
data_weather.head()

## OLD PREPROCESSING

### Reading in the data from the S3 bucket (don't forget to pip install boto3)

In [ ]:
# # meteo data
# Q1_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q1.csv')
# Q2_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q2.csv')
# Q3_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q3.csv')
# Q4_2022 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_2022Q4.csv')

In [ ]:
# REMARK: this is the 'old' noise data, don't run this

# noise data
# exp40_naamse35 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# exp40_naamse57 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# exp40_naamse62 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# exp40_calvarie = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# exp40_naamse81 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# exp40_park = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# exp40_kiosk = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# exp40_vrijt = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# exp40_his = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_40/csv_results_40_303910_mp-04-his-hears.csv', header=0, sep=';')

# exp41_naamse35 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# exp41_naamse57 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# exp41_naamse62 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# exp41_calvarie = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# exp41_naamse81 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# exp41_park = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# exp41_kiosk = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# exp41_vrijt = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# exp41_his = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_41/csv_results_41_303910_mp-04-his-hears.csv', header=0, sep=';')

# exp42_naamse35 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# exp42_naamse57 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# exp42_naamse62 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# exp42_calvarie = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# exp42_naamse81 = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# exp42_park = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# exp42_kiosk = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# exp42_vrijt = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# exp42_his = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/export_42/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')


In [ ]:
# noise_columns = ["#object_id", "description", "result_timestamp", "lamax", "laeq"]
# naamse35_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';', usecols=noise_columns)

In [ ]:
# # updated noise data - January
# naamse35_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_jan = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - February
# naamse35_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_feb = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - March
# naamse35_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_mar = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - April
# naamse35_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_apr = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - May
# naamse35_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_may = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - June
# naamse35_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_jun = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - July
# naamse35_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_jul = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - August
# naamse35_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_aug = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - September
# naamse35_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_sep = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - October
# naamse35_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_oct = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - November
# naamse35_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_nov = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_303910_mp-04-his-hears.csv', header=0, sep=';')

In [ ]:
# # updated noise data - December
# naamse35_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255439_mp-01-naamsestraat-35-maxim.csv', header=0, sep=';')
# naamse57_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255440_mp-02-naamsestraat-57-xior.csv', header=0, sep=';')
# naamse62_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255441_mp-03-naamsestraat-62-taste.csv', header=0, sep=';')
# calvarie_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255442_mp-05-calvariekapel-ku-leuven.csv', header=0, sep=';')
# park_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255443_mp-06-parkstraat-2-la-filosovia.csv', header=0, sep=';')
# naamse81_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255444_mp-07-naamsestraat-81.csv', header=0, sep=';')
# kiosk_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255445_mp-08-kiosk-stadspark.csv', header=0, sep=';')
# vrijt_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_280324_mp08bis---vrijthof.csv', header=0, sep=';')
# his_dec = pd.read_csv('https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_303910_mp-04-his-hears.csv', header=0, sep=';')

### Combining and aggregating the meteo data

In [ ]:
# # combine meteo dataset 
# meteocombined = pd.concat([Q1_2022, Q2_2022, Q3_2022, Q4_2022], axis=0)
# meteocombined.head()

In [ ]:
# check for missing values in each column
# print(meteocombined.isnull().sum())

In [ ]:
# # aggregate meteo data by day
# avg_meteo_combined = meteocombined.groupby(['Year','Month', 'Day']).mean()
# avg_meteo_combined = avg_meteo_combined.reset_index()
# avg_meteo_combined.head()


In [ ]:
# month_max_value = avg_meteo_combined['Month'].max()
# print(f"This combined meteo dataset contains the weather data for all {month_max_value} months.")

### Combining and aggregating the noise data

- January

In [ ]:
# # combine noise data for January together
# noise_jan_combined = pd.concat([naamse35_jan, naamse57_jan, naamse62_jan, calvarie_jan, park_jan, naamse81_jan, kiosk_jan, vrijt_jan, his_jan], axis=0)
# noise_jan_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_jan_combined['result_timestamp'] = pd.to_datetime(noise_jan_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_jan_combined['result_date'] = noise_jan_combined['result_timestamp'].dt.date
# noise_jan_combined['result_month'] = noise_jan_combined['result_timestamp'].dt.month
# noise_jan_combined['result_day'] = noise_jan_combined['result_timestamp'].dt.day
# noise_jan_combined['result_hour'] = noise_jan_combined['result_timestamp'].dt.hour
# noise_jan_combined['result_minute'] = noise_jan_combined['result_timestamp'].dt.minute

# noise_jan_combined.head()

In [ ]:
# # aggregate the data by day
# avg_jan_combined = noise_jan_combined.groupby(['result_date','description']).mean()
# avg_jan_combined = avg_jan_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_jan_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_jan_combined.head(150)

In [ ]:
# check for missing values in each column
# print(avg_jan_combined.isnull().sum())

- February

In [ ]:
# # combine noise data for February together
# noise_feb_combined = pd.concat([naamse35_feb, naamse57_feb, naamse62_feb, calvarie_feb, park_feb, naamse81_feb, kiosk_feb, vrijt_feb, his_feb], axis=0)
# noise_feb_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_feb_combined['result_timestamp'] = pd.to_datetime(noise_feb_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_feb_combined['result_date'] = noise_feb_combined['result_timestamp'].dt.date
# noise_feb_combined['result_month'] = noise_feb_combined['result_timestamp'].dt.month
# noise_feb_combined['result_day'] = noise_feb_combined['result_timestamp'].dt.day
# noise_feb_combined['result_hour'] = noise_feb_combined['result_timestamp'].dt.hour
# noise_feb_combined['result_minute'] = noise_feb_combined['result_timestamp'].dt.minute

# noise_feb_combined.head()

In [ ]:
# # aggregate the data by day
# avg_feb_combined = noise_feb_combined.groupby(['result_date','description']).mean()
# avg_feb_combined = avg_feb_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_feb_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_feb_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_feb_combined.isnull().sum())

- March

In [ ]:
# # combine noise data for March together
# noise_mar_combined = pd.concat([naamse35_mar, naamse57_mar, naamse62_mar, calvarie_mar, park_mar, naamse81_mar, kiosk_mar, vrijt_mar, his_mar], axis=0)
# noise_mar_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_mar_combined['result_timestamp'] = pd.to_datetime(noise_mar_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_mar_combined['result_date'] = noise_mar_combined['result_timestamp'].dt.date
# noise_mar_combined['result_month'] = noise_mar_combined['result_timestamp'].dt.month
# noise_mar_combined['result_day'] = noise_mar_combined['result_timestamp'].dt.day
# noise_mar_combined['result_hour'] = noise_mar_combined['result_timestamp'].dt.hour
# noise_mar_combined['result_minute'] = noise_mar_combined['result_timestamp'].dt.minute

# noise_mar_combined.head()

In [ ]:
# # aggregate the data by day
# avg_mar_combined = noise_mar_combined.groupby(['result_date','description']).mean()
# avg_mar_combined = avg_mar_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_mar_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_mar_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_mar_combined.isnull().sum())

- April 

In [ ]:
# # combine noise data for April together
# noise_apr_combined = pd.concat([naamse35_apr, naamse57_apr, naamse62_apr, calvarie_apr, park_apr, naamse81_apr, kiosk_apr, vrijt_apr, his_apr], axis=0)
# noise_apr_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_apr_combined['result_timestamp'] = pd.to_datetime(noise_apr_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_apr_combined['result_date'] = noise_apr_combined['result_timestamp'].dt.date
# noise_apr_combined['result_month'] = noise_apr_combined['result_timestamp'].dt.month
# noise_apr_combined['result_day'] = noise_apr_combined['result_timestamp'].dt.day
# noise_apr_combined['result_hour'] = noise_apr_combined['result_timestamp'].dt.hour
# noise_apr_combined['result_minute'] = noise_apr_combined['result_timestamp'].dt.minute

# noise_apr_combined.head()

In [ ]:
# # aggregate the data by day
# avg_apr_combined = noise_apr_combined.groupby(['result_date','description']).mean()
# avg_apr_combined = avg_apr_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_apr_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_apr_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_apr_combined.isnull().sum())

- May 

In [ ]:
# # combine noise data for May together
# noise_may_combined = pd.concat([naamse35_may, naamse57_may, naamse62_may, calvarie_may, park_may, naamse81_may, kiosk_may, vrijt_may, his_may], axis=0)
# noise_may_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_may_combined['result_timestamp'] = pd.to_datetime(noise_may_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_may_combined['result_date'] = noise_may_combined['result_timestamp'].dt.date
# noise_may_combined['result_month'] = noise_may_combined['result_timestamp'].dt.month
# noise_may_combined['result_day'] = noise_may_combined['result_timestamp'].dt.day
# noise_may_combined['result_hour'] = noise_may_combined['result_timestamp'].dt.hour
# noise_may_combined['result_minute'] = noise_may_combined['result_timestamp'].dt.minute

# noise_may_combined.head()

In [ ]:
# # aggregate the data by day
# avg_may_combined = noise_may_combined.groupby(['result_date','description']).mean()
# avg_may_combined = avg_may_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_may_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_may_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_may_combined.isnull().sum())

- June

In [ ]:
# # combine noise data for June together
# noise_jun_combined = pd.concat([naamse35_jun, naamse57_jun, naamse62_jun, calvarie_jun, park_jun, naamse81_jun, kiosk_jun, vrijt_jun, his_jun], axis=0)
# noise_jun_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_jun_combined['result_timestamp'] = pd.to_datetime(noise_jun_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_jun_combined['result_date'] = noise_jun_combined['result_timestamp'].dt.date
# noise_jun_combined['result_month'] = noise_jun_combined['result_timestamp'].dt.month
# noise_jun_combined['result_day'] = noise_jun_combined['result_timestamp'].dt.day
# noise_jun_combined['result_hour'] = noise_jun_combined['result_timestamp'].dt.hour
# noise_jun_combined['result_minute'] = noise_jun_combined['result_timestamp'].dt.minute

# noise_jun_combined.head()

In [ ]:
# aggregate the data by day
# avg_jun_combined = noise_jun_combined.groupby(['result_date','description']).mean()
# avg_jun_combined = avg_jun_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_jun_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_jun_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_jun_combined.isnull().sum())

- July

In [ ]:
# # combine noise data for July together
# noise_jul_combined = pd.concat([naamse35_jul, naamse57_jul, naamse62_jul, calvarie_jul, park_jul, naamse81_jul, kiosk_jul, vrijt_jul, his_jul], axis=0)
# noise_jul_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_jul_combined['result_timestamp'] = pd.to_datetime(noise_jul_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_jul_combined['result_date'] = noise_jul_combined['result_timestamp'].dt.date
# noise_jul_combined['result_month'] = noise_jul_combined['result_timestamp'].dt.month
# noise_jul_combined['result_day'] = noise_jul_combined['result_timestamp'].dt.day
# noise_jul_combined['result_hour'] = noise_jul_combined['result_timestamp'].dt.hour
# noise_jul_combined['result_minute'] = noise_jul_combined['result_timestamp'].dt.minute

# noise_jul_combined.head()

In [ ]:
# # aggregate the data by day
# avg_jul_combined = noise_jul_combined.groupby(['result_date','description']).mean()
# avg_jul_combined = avg_jul_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_jul_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_jul_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_jul_combined.isnull().sum())

- August

In [ ]:
# # combine noise data for August together
# noise_aug_combined = pd.concat([naamse35_aug, naamse57_aug, naamse62_aug, calvarie_aug, park_aug, naamse81_aug, kiosk_aug, vrijt_aug, his_aug], axis=0)
# noise_aug_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_aug_combined['result_timestamp'] = pd.to_datetime(noise_aug_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_aug_combined['result_date'] = noise_aug_combined['result_timestamp'].dt.date
# noise_aug_combined['result_month'] = noise_aug_combined['result_timestamp'].dt.month
# noise_aug_combined['result_day'] = noise_aug_combined['result_timestamp'].dt.day
# noise_aug_combined['result_hour'] = noise_aug_combined['result_timestamp'].dt.hour
# noise_aug_combined['result_minute'] = noise_aug_combined['result_timestamp'].dt.minute

# noise_aug_combined.head()

In [ ]:
# # aggregate the data by day
# avg_aug_combined = noise_aug_combined.groupby(['result_date','description']).mean()
# avg_aug_combined = avg_aug_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_aug_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_aug_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_aug_combined.isnull().sum())

- September

In [ ]:
# # combine noise data for September together
# noise_sep_combined = pd.concat([naamse35_sep, naamse57_sep, naamse62_sep, calvarie_sep, park_sep, naamse81_sep, kiosk_sep, vrijt_sep, his_sep], axis=0)
# noise_sep_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_sep_combined['result_timestamp'] = pd.to_datetime(noise_sep_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_sep_combined['result_date'] = noise_sep_combined['result_timestamp'].dt.date
# noise_sep_combined['result_month'] = noise_sep_combined['result_timestamp'].dt.month
# noise_sep_combined['result_day'] = noise_sep_combined['result_timestamp'].dt.day
# noise_sep_combined['result_hour'] = noise_sep_combined['result_timestamp'].dt.hour
# noise_sep_combined['result_minute'] = noise_sep_combined['result_timestamp'].dt.minute

# noise_sep_combined.head()

In [ ]:
# # aggregate the data by day
# avg_sep_combined = noise_sep_combined.groupby(['result_date','description']).mean()
# avg_sep_combined = avg_sep_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_sep_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_sep_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_sep_combined.isnull().sum())

- October

In [ ]:
# # combine noise data for Octber together
# noise_oct_combined = pd.concat([naamse35_oct, naamse57_oct, naamse62_oct, calvarie_oct, park_oct, naamse81_oct, kiosk_oct, vrijt_oct, his_oct], axis=0)
# noise_oct_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_oct_combined['result_timestamp'] = pd.to_datetime(noise_oct_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_oct_combined['result_date'] = noise_oct_combined['result_timestamp'].dt.date
# noise_oct_combined['result_month'] = noise_oct_combined['result_timestamp'].dt.month
# noise_oct_combined['result_day'] = noise_oct_combined['result_timestamp'].dt.day
# noise_oct_combined['result_hour'] = noise_oct_combined['result_timestamp'].dt.hour
# noise_oct_combined['result_minute'] = noise_oct_combined['result_timestamp'].dt.minute

# noise_oct_combined.head()

In [ ]:
# # aggregate the data by day
# avg_oct_combined = noise_oct_combined.groupby(['result_date','description']).mean()
# avg_oct_combined = avg_oct_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_oct_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_oct_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_oct_combined.isnull().sum())

- November

In [ ]:
# # combine noise data for November together
# noise_nov_combined = pd.concat([naamse35_nov, naamse57_nov, naamse62_nov, calvarie_nov, park_nov, naamse81_nov, kiosk_nov, vrijt_nov, his_nov], axis=0)
# noise_nov_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_nov_combined['result_timestamp'] = pd.to_datetime(noise_nov_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_nov_combined['result_date'] = noise_nov_combined['result_timestamp'].dt.date
# noise_nov_combined['result_month'] = noise_nov_combined['result_timestamp'].dt.month
# noise_nov_combined['result_day'] = noise_nov_combined['result_timestamp'].dt.day
# noise_nov_combined['result_hour'] = noise_nov_combined['result_timestamp'].dt.hour
# noise_nov_combined['result_minute'] = noise_nov_combined['result_timestamp'].dt.minute

# noise_nov_combined.head()

In [ ]:
# # aggregate the data by day
# avg_nov_combined = noise_nov_combined.groupby(['result_date','description']).mean()
# avg_nov_combined = avg_nov_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_nov_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_nov_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_nov_combined.isnull().sum())

- December

In [ ]:
# # combine noise data for December together
# noise_dec_combined = pd.concat([naamse35_dec, naamse57_dec, naamse62_dec, calvarie_dec, park_dec, naamse81_dec, kiosk_dec, vrijt_dec, his_dec], axis=0)
# noise_dec_combined.head()

In [ ]:
# # extract the date, month, hour, minute of "result_timestamp"
# noise_dec_combined['result_timestamp'] = pd.to_datetime(noise_dec_combined['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')


# noise_dec_combined['result_date'] = noise_dec_combined['result_timestamp'].dt.date
# noise_dec_combined['result_month'] = noise_dec_combined['result_timestamp'].dt.month
# noise_dec_combined['result_day'] = noise_dec_combined['result_timestamp'].dt.day
# noise_dec_combined['result_hour'] = noise_dec_combined['result_timestamp'].dt.hour
# noise_dec_combined['result_minute'] = noise_dec_combined['result_timestamp'].dt.minute

# noise_dec_combined.head()

In [ ]:
# # aggregate the data by day
# avg_dec_combined = noise_dec_combined.groupby(['result_date','description']).mean()
# avg_dec_combined = avg_dec_combined.reset_index()
# columns_to_drop = ['result_hour', 'result_minute']
# avg_dec_combined.drop(columns_to_drop, axis=1, inplace=True)
# avg_dec_combined.head(150)

In [ ]:
# # check for missing values in each column
# print(avg_dec_combined.isnull().sum())

Combining monthly noise level datasets into a yearly dataset

In [ ]:
# List of the monthly datasets
datasets = [avg_jan_combined, avg_feb_combined, avg_mar_combined, avg_apr_combined, avg_may_combined, avg_jun_combined, avg_jul_combined, avg_aug_combined, avg_sep_combined, avg_oct_combined, avg_nov_combined, avg_dec_combined]

# Concatenate the datasets vertically
avg_year_combined = pd.concat(datasets, ignore_index=True)

# Sort the combined dataset by 'result_date' in ascending order
avg_year_combined.sort_values(by='result_date', inplace=True)

# Reset the index of the combined dataset
avg_year_combined.reset_index(drop=True, inplace=True)

# Display the combined and sorted yearly dataset
avg_year_combined.head(2000)
